In [1]:
import pandas as pd
import numpy as np
import cv2
import sys
import caffe
import pickle
from caffe.proto import caffe_pb2
import random

In [2]:
def INBS(S,e, epsilon, accuracies_NBS, accuracies_IBS_1, P, longitud1):
    global longitud
    global P1
    global accuracies_IBS
    P1=P
    longitud=longitud1
    accuracies_IBS=accuracies_IBS_1
    n=len(S)
    nal=len(S)
    k=12*np.log(n)
    c=3*np.log(n)
    h=np.ceil(np.log2(n-1))
    c1=0
    k1=0
    ll=1
    uu=np.power(2, h)+1
    SS = np.zeros(n)
    while (k1<k) and (c1<c):
        k1=k1+1
        if (uu-ll)>1:
            ll1=int(ll)
            uu1=int(uu)
            mm=int(np.ceil((ll+uu)/2.0))
            mm1=mm
            if (uu>n):
                uu1=n
                if mm>n:
                    mm1=n
                if ll>n:
                    ll1=n
            SS[ll1-1]=1
            SS[uu1-1]=1
            SS[mm1-1]=1
            
            if compare2(S[ll1-1], e)==1 and compare2(S[uu1-1], e)==2:
                if compare2(S[mm1-1], e)==1:    
                    ll=int(np.ceil((ll+uu)/float(2)))
                else:
                    uu=int(np.ceil((ll+uu)/float(2)))
            else:
                h1=np.log2(uu-ll)
                if h1<h:
                    if np.remainder(ll-1, np.power(2, h1+1))==0:
                        uu=ll+np.power(2, h1+1)
                    else:
                        ll=uu-np.power(2, h1+1)
        else:
            if uu>n:
                uu1=n
                if ll>n:
                    ll1=n
                
            if compare2(S[ll1-1], e)==1 and compare2(S[uu1-1], e)==2:
                c1=c1+1
            else:
                if c1==0:
                    if np.remainder(ll-1, 2)==0:
                        uu=ll+2
                    else:
                        ll=uu-2
                else:
                    c1=c1-1

    if c1>=c:
        x=ll-1
    else:
        size=0
        for i in np.arange(n):
            if SS[i]==1:
                size=size+1
        SSS=np.zeros(size)
        index=0
        for i in np.arange(n):
            if SS[i]==1:
                SSS[index]=i
                index=index+1
                
        #Binary search
        ll=1
        uu=size
        while ((uu-ll)>0):
            
            mm=np.ceil((ll+uu)/2.0)
  
            if S[int(SSS[int(mm)-1])]==16:
                hp_wins=0
            elif S[int(SSS[int(mm)-1])]==40:
                hp_wins=1
            else:
                ind2=np.random.permutation(longitud[S[int(SSS[int(mm)-1])]])
                ind2=ind2[0:accuracies_NBS[S[int(SSS[int(mm)-1])]]]

                img2=P1[S[int(SSS[int(mm)-1])]][ind2]
                hp_wins=dataset_and_compare(img2, e)

            if (.5-epsilon)<=hp_wins and hp_wins<=(.5+epsilon):
                x=SSS[int(mm)-1]
                break
            else:
                if hp_wins<(.5-epsilon):
                    if mm==1:
                        x=SSS[0]
                        break
                    ll=np.ceil((ll+uu)/2.0)
                else:
                    if (mm==size):
                        x=SSS[size-1]
                        break
                    uu=np.ceil((ll+uu)/float(2))
                if (uu-ll==1) & (mm==uu):
                    x=SSS[int(ll)-1]
                    break
        if (uu-ll)==0:
            x=SSS[int(ll)-1]
    
    return S[int(x)]


def dataset_and_compare(img2,ej):
    
  
    img2=img2.values
    images=np.zeros((len(img2), 6, 224, 224))
    dim=len(img2)
    index=0
    for i in img2:
        img_path_1='./Anchors/Images/'+i+'.jpg' 
        img1 = cv2.imread(img_path_1, cv2.IMREAD_COLOR)
        if img1==None:
            dim=dim-1
            images = images[0:dim, :, :]
            continue
        
        img=np.concatenate((img1, ej), axis=2)
        img = img.transpose((2,0,1))
        images[index,:,:,:]=img
        index=index+1
    out = net0.forward_all(pair_data = images )
    bla=sum(out['prob1']>0.5)/float(len(out['prob1']))
    return bla


def compare2(ei,ej):
    if ei==16:
        y=1
        return y
    elif ei==40:
        y=2
        return y
    
    
    global P1
    global longitud
    global accuracies_IBS

    ind1=np.random.permutation(longitud[ei]-1)
    ind1=ind1[0:accuracies_IBS[ei]]
    
    img2=P1[ei][ind1]
    
    hp=dataset_and_compare(img2, ej)
    if (hp<=.5):
        y=1
    else:
        y=2
    return y    
                 

In [3]:
with open('./Anchors/table_anchors.pkl', 'rb') as f:
    [df, longitud]=pickle.load(f)

epsilon=0.03
budget_IBS=50
budget_NBS=50
GPU_ID=0

caffe_root = './caffe/'  
sys.path.insert(0, caffe_root + 'python')
caffe.set_mode_gpu()
caffe.set_device(GPU_ID)
model_def = './Deployment_prototxt_files/AgeNet-based_Mode_II.prototxt'
model_weights0 = './Models/AgeNet-based_Mode_II.caffemodel'


net0 = caffe.Net(model_def,      # defines the structure of the model
                model_weights0,  # contains the trained weights
                caffe.TEST)     


roc_values=np.array([0.797797, 0.712388, 0.744831, 0.788210])
roc_values=(1-roc_values)/sum(1-roc_values)

accuracies={}
accuracies_IBS={}
accuracies_NBS={}
accuracies[18.5]=roc_values[0]
accuracies[25]=roc_values[1]
accuracies[30]=roc_values[2]
accuracies[35]=roc_values[3]

for key in accuracies:    
    accuracies_IBS[key] =  int(budget_IBS*accuracies[key])
    accuracies_NBS[key] =  int(budget_NBS*accuracies[key])
    
S=[16, 18.5, 25, 30, 35, 40]


df4_neutral = pd.DataFrame(columns=('Athlete', 'Ground truth (BMI)', 'Prediction'))
Names=['LIM HYOJUN','DAVID GLEIRSCHER']
BMI=[22.67, 26.87]

for tra in range(2):
            img_path_1='./Testing_images/'+str(tra)+'.jpg'
            img1= cv2.imread(img_path_1, cv2.IMREAD_COLOR)
            pred=INBS(S,img1, epsilon, accuracies_NBS, accuracies_IBS, df, longitud)
            df4_neutral.loc[tra]=[Names[tra], BMI[tra], pred]

df4_neutral.loc[(df4_neutral['Prediction'] ==16),'Prediction'] = 'Underweight (16-18.5)'
df4_neutral.loc[(df4_neutral['Prediction'] ==18.5),'Prediction'] = 'Normal (18.5-25)'
df4_neutral.loc[(df4_neutral['Prediction'] ==25),'Prediction'] = 'Overweight (25-30)'
df4_neutral.loc[(df4_neutral['Prediction'] ==30),'Prediction'] = 'Moderately obese (30-35)'
df4_neutral.loc[(df4_neutral['Prediction'] ==35),'Prediction'] = 'Severely obese (35-40)'

/home/lpolania/enter/lib/python2.7/site-packages/ipykernel/__main__.py:129: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


In [4]:
#Results
df4_neutral

,Athlete,Ground truth (BMI),Prediction
0,LIM HYOJUN,22.67,Normal (18.5-25)
1,DAVID GLEIRSCHER,26.87,Overweight (25-30)
